<a href="https://colab.research.google.com/github/remphanstar/WanBook/blob/main/Welcome_To_Colab.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
#@title Cell (1) WAN2GP Setup Introduction & Enhanced Platform Detection v3.1 (Colab venv Enforced)

import sys
from IPython.display import display, HTML, Markdown
import os

def jupyter_lab_title(title):
    if "google.colab" not in sys.modules:
        display(Markdown(f"## {title}"))

jupyter_lab_title("WAN2GP Setup Introduction & Enhanced Platform Detection")

def detect_platform():
    """Enhanced platform detection with comprehensive indicators"""

    # Lightning AI detection - multiple indicators for reliability
    lightning_indicators = [
        "lightning" in str(sys.executable).lower(),
        "teamspace-studios" in os.getcwd(),
        "LIGHTNING_CLOUDSPACE_HOST" in os.environ,
        "LIGHTNING_CLOUDSPACE_ID" in os.environ,
        "commands/python" in str(sys.executable),
        "/home/zeus/miniconda3/envs/cloudspace" in str(sys.executable),
        os.path.exists("/teamspace"),
        os.path.exists("/commands")
    ]

    # Google Colab detection
    colab_indicators = [
        "google.colab" in sys.modules,
        "/content" in os.getcwd()
    ]

    # Vast.AI detection
    vast_indicators = [
        "VAST_CONTAINER_LABEL" in os.environ,
        "/workspace" in os.getcwd(),
        "vast" in os.environ.get("HOSTNAME", "").lower()
    ]

    if any(lightning_indicators):
        return "Lightning AI"
    elif any(colab_indicators):
        return "Google Colab"
    elif any(vast_indicators):
        return "Vast.AI/Generic"
    else:
        return "Vast.AI/Generic"

def get_platform_commands(platform):
    """Get platform-specific pip and python commands - ENFORCED VENV FOR COLAB"""
    if platform == "Lightning AI":
        return "pip", "python", False  # (pip_cmd, python_cmd, use_venv)
    elif platform == "Google Colab":
        # ENFORCED: Always use venv for Google Colab
        return ".venv/bin/pip", ".venv/bin/python", True
    else:  # Vast.AI/Generic
        return ".venv/bin/pip", ".venv/bin/python", True

# Detect current platform
current_platform = detect_platform()
pip_cmd, python_cmd, use_venv = get_platform_commands(current_platform)

# Display platform information
display(HTML(f"""
<div style="font-family: Arial, sans-serif; background: linear-gradient(135deg, #667eea 0%, #764ba2 100%);
           color: white; padding: 20px; border-radius: 10px; text-align: center; margin: 10px 0;
           box-shadow: 0 4px 15px rgba(0,0,0,0.2);">
    <h1>🚀 Wan2GP Full Setup Notebook</h1>
    <p>A cross-platform Jupyter notebook (Colab, Lightning AI, Vast.ai) that installs Wan2GP,
       common LoRA packs, and optional performance extras (FlashAttention 2, SageAttention, xFormers).
       It accelerates all downloads with <strong>aria2c</strong> for maximum speed.</p>
</div>

<div style="background-color: #f8f9fa; padding: 20px; border-radius: 8px; border: 1px solid #dee2e6; margin: 15px 0;">
    <div style="background-color: #007bff; color: white; padding: 15px; border-radius: 5px; text-align: center; margin-bottom: 20px;">
        <h3 style="margin: 0; color: white;">🎯 Platform Detection Results</h3>
        <p style="margin: 5px 0 0 0; color: white;">Currently Running on: <strong style="color: #28a745; background-color: white; padding: 2px 6px; border-radius: 3px;">{current_platform}</strong></p>
        <p style="margin: 5px 0 0 0; color: white;">Virtual Environment: <strong>{"Yes" if use_venv else "No (Lightning AI)"}</strong></p>
        <p style="margin: 5px 0 0 0; color: white;">Pip Command: <strong>{pip_cmd}</strong></p>
        <p style="margin: 5px 0 0 0; color: white;">Python Command: <strong>{python_cmd}</strong></p>
    </div>
</div>
"""))

print("📋 Platform Detection Debug Info:")
print(f"🔍 - Detected Platform: {current_platform}")
print(f"🐍 - Python Executable: {sys.executable}")
print(f"📁 - Current Working Directory: {os.getcwd()}")
print(f"🌐 - Google Colab Check: {'google.colab' in sys.modules}")
print(f"⚡ - Lightning Environment Variables: {[key for key in os.environ.keys() if 'LIGHTNING' in key]}")
print(f"🔧 - Virtual Environment Usage: {use_venv}")
print(f"📦 - Pip Command: {pip_cmd}")
print(f"🐍 - Python Command: {python_cmd}")

# CRITICAL: Enforce venv for Google Colab
if current_platform == "Google Colab" and not use_venv:
    print("🚨 CRITICAL: Detected Google Colab but venv is disabled - FORCE ENABLING")
    use_venv = True
    pip_cmd = ".venv/bin/pip"
    python_cmd = ".venv/bin/python"
    print("✅ ENFORCED: Google Colab now using venv commands")
elif current_platform == "Google Colab":
    print("✅ CONFIRMED: Google Colab correctly configured for venv usage")


📋 Platform Detection Debug Info:
🔍 - Detected Platform: Google Colab
🐍 - Python Executable: /usr/bin/python3
📁 - Current Working Directory: /content
🌐 - Google Colab Check: True
⚡ - Lightning Environment Variables: []
🔧 - Virtual Environment Usage: True
📦 - Pip Command: .venv/bin/pip
🐍 - Python Command: .venv/bin/python
✅ CONFIRMED: Google Colab correctly configured for venv usage


In [2]:
#@title Cell (2) Complete System + Python Environment Setup - Cross-Platform v3.2 (venv Enforced for Colab)

import sys
import os
import subprocess
import shutil
import time
from pathlib import Path

def jupyter_lab_title(title):
    if "google.colab" not in sys.modules:
        from IPython.display import display, Markdown
        display(Markdown(f"## {title}"))

jupyter_lab_title("Complete System + Python Environment Setup")

class CompleteEnvironmentSetup:
    def __init__(self):
        # Use global platform configuration from Cell 1
        try:
            self.current_platform = current_platform
            self.pip_cmd = pip_cmd
            self.python_cmd = python_cmd
            self.use_venv = use_venv
        except NameError:
            print("⚠️ Platform detection variables not found. Using fallback detection.")
            self.detect_platform_fallback()

        # CRITICAL ENFORCEMENT: Ensure venv for Google Colab
        self.enforce_colab_venv()

        self.setup_phases = [
            ("System Packages", self.install_system_packages),
            ("Directory Protection", self.setup_directory_protection),
            ("Repository Clone", self.clone_repository),
            ("Virtual Environment", self.setup_virtual_environment),
            ("PyTorch Installation", self.install_pytorch),
            ("Requirements", self.install_requirements),
            ("Environment Verification", self.verify_complete_setup),
        ]

        self.current_phase = 0
        self.total_phases = len(self.setup_phases)

    def enforce_colab_venv(self):
        """CRITICAL: Enforce venv usage for Google Colab"""
        if self.current_platform == "Google Colab":
            if not self.use_venv or self.pip_cmd != ".venv/bin/pip" or self.python_cmd != ".venv/bin/python":
                print("🚨 ENFORCING: Google Colab MUST use venv - correcting configuration")
                self.use_venv = True
                self.pip_cmd = ".venv/bin/pip"
                self.python_cmd = ".venv/bin/python"
                print("✅ ENFORCED: Google Colab venv configuration corrected")
            else:
                print("✅ VERIFIED: Google Colab correctly using venv configuration")

    def detect_platform_fallback(self):
        """Fallback platform detection if Cell 1 wasn't run - ENFORCES VENV FOR COLAB"""
        if "google.colab" in sys.modules:
            self.current_platform = "Google Colab"
            # ENFORCED: Always venv for Colab
            self.pip_cmd, self.python_cmd, self.use_venv = ".venv/bin/pip", ".venv/bin/python", True
        elif any(["lightning" in str(sys.executable).lower(), "teamspace" in os.getcwd()]):
            self.current_platform = "Lightning AI"
            self.pip_cmd, self.python_cmd, self.use_venv = "pip", "python", False
        else:
            self.current_platform = "Generic"
            self.pip_cmd, self.python_cmd, self.use_venv = ".venv/bin/pip", ".venv/bin/python", True

    def update_progress(self, phase_name, status="in_progress"):
        """Update setup progress with visual indicators"""
        progress_percent = (self.current_phase / self.total_phases) * 100
        status_icons = {
            "in_progress": "🔄",
            "success": "✅",
            "failed": "❌"
        }

        print(f"\n[{self.current_phase + 1}/{self.total_phases}] {status_icons[status]} {phase_name}")
        print(f"Progress: {progress_percent:.1f}% | Platform: {self.current_platform}")
        print(f"🔧 Using: {self.pip_cmd} | {self.python_cmd} | venv: {self.use_venv}")
        print("=" * 60)

    def run_command_safely(self, command, description, timeout=300):
        """Execute command with comprehensive error handling"""
        try:
            result = subprocess.run(
                command,
                capture_output=True,
                text=True,
                timeout=timeout
            )

            if result.returncode == 0:
                print(f"✅ {description} completed successfully")
                return True, result.stdout
            else:
                print(f"❌ {description} failed (exit code {result.returncode})")
                if result.stderr:
                    # Check for specific ensurepip failure
                    if "ensurepip" in result.stderr:
                        print(f"⚠️ ensurepip module failure detected")
                        return "ensurepip_failure", result.stderr
                    print(f"   Error: {result.stderr.strip()[:200]}")
                return False, result.stderr

        except subprocess.TimeoutExpired:
            print(f"⏰ {description} timed out after {timeout}s")
            return False, f"Timeout after {timeout}s"
        except Exception as e:
            print(f"💥 {description} crashed: {e}")
            return False, str(e)

    def install_system_packages(self):
        """Install system packages - same for all platforms"""
        print("🔧 Installing system packages...")

        if not shutil.which("apt-get"):
            print("📦 apt-get not found. Skipping system package installation.")
            return True

        try:
            print("📦 Updating package lists...")
            subprocess.run(["sudo", "apt-get", "update", "-qq"], check=True, timeout=60)

            print("🔧 Installing aria2, git, build-essential, and wget...")
            subprocess.run(["sudo", "apt-get", "install", "-y", "aria2", "git", "build-essential", "wget"], check=True, timeout=180)

            print("✅ System packages installed successfully")
            return True
        except subprocess.CalledProcessError as e:
            print(f"❌ System package installation failed: {e}")
            return False
        except subprocess.TimeoutExpired:
            print("⏰ System package installation timed out")
            return False

    def setup_directory_protection(self):
        """Critical directory protection to prevent recursive cloning"""
        print("🛡️ Setting up directory protection...")

        current_dir = os.getcwd()
        print(f"📍 Current directory: {current_dir}")

        # Check if we're already inside a Wan2GP directory
        if "Wan2GP" in current_dir:
            print("⚠️ Already inside Wan2GP directory - navigating to safe location")

            # Navigate to content root (Colab) or appropriate base directory
            if self.current_platform == "Google Colab":
                safe_dir = "/content"
            elif self.current_platform == "Lightning AI":
                safe_dir = os.path.expanduser("~")
            else:
                safe_dir = os.path.expanduser("~")

            try:
                os.chdir(safe_dir)
                print(f"📁 Moved to safe directory: {os.getcwd()}")
            except Exception as e:
                print(f"❌ Failed to navigate to safe directory: {e}")
                return False

        # Check for existing Wan2GP directory and handle it
        repo_path = os.path.join(os.getcwd(), "Wan2GP")
        if os.path.exists(repo_path):
            print(f"📁 Found existing Wan2GP directory at: {repo_path}")

            # Check if it looks like a valid repository
            if os.path.exists(os.path.join(repo_path, ".git")):
                print("✅ Existing directory appears to be a valid git repository")
                # Check if it's the correct repository
                try:
                    os.chdir(repo_path)
                    result = subprocess.run(["git", "remote", "get-url", "origin"],
                                          capture_output=True, text=True, timeout=10)
                    if result.returncode == 0 and "Wan2GP" in result.stdout:
                        print("✅ Existing repository is correct - using it")
                        return True
                    else:
                        print("⚠️ Existing repository is not the correct one")
                        os.chdir("..")
                except Exception as e:
                    print(f"⚠️ Could not verify existing repository: {e}")
                    os.chdir("..")

            # If we reach here, remove the problematic directory
            print("🗑️ Removing problematic existing directory...")
            try:
                shutil.rmtree(repo_path)
                print("✅ Problematic directory removed")
            except Exception as e:
                print(f"❌ Failed to remove existing directory: {e}")
                # Try to rename it instead
                try:
                    backup_name = f"Wan2GP_backup_{int(time.time())}"
                    os.rename(repo_path, backup_name)
                    print(f"✅ Renamed problematic directory to {backup_name}")
                except Exception as e2:
                    print(f"❌ Could not even rename directory: {e2}")
                    return False

        print("✅ Directory protection setup complete")
        return True

    def clone_repository(self):
        """Clone WAN2GP repository with enhanced protection"""
        REPO_DIR = "Wan2GP"
        REPO_URL = "https://github.com/deepbeepmeep/Wan2GP.git"

        print("📂 Setting up WAN2GP repository...")
        print(f"📍 Working from: {os.getcwd()}")

        # Final check - ensure we're not in a nested situation
        current_path = os.getcwd()
        if current_path.count("Wan2GP") > 0:
            print("🚨 CRITICAL: Still in Wan2GP directory path!")
            print("🔄 Attempting emergency navigation...")

            # Emergency navigation
            if self.current_platform == "Google Colab":
                emergency_path = "/content"
            else:
                emergency_path = os.path.expanduser("~")

            try:
                os.chdir(emergency_path)
                print(f"🆘 Emergency navigation successful: {os.getcwd()}")
            except Exception as e:
                print(f"💥 Emergency navigation failed: {e}")
                return False

        if not os.path.exists(REPO_DIR):
            print(f"📥 Cloning repository from {REPO_URL}...")
            try:
                subprocess.run(["git", "clone", "--depth", "1", REPO_URL], check=True, timeout=120)
                print("✅ Repository cloned successfully")
            except subprocess.CalledProcessError as e:
                print(f"❌ Failed to clone repository: {e}")
                return False
            except subprocess.TimeoutExpired:
                print("⏰ Repository clone timed out")
                return False
        else:
            print(f"📁 Directory {REPO_DIR} already exists")

        try:
            os.chdir(REPO_DIR)
            final_path = os.getcwd()
            print(f"📁 Changed to directory: {final_path}")

            # Verify we're in the right place
            if final_path.count("Wan2GP") > 1:
                print("🚨 WARNING: Detected nested Wan2GP directories!")
                print(f"🔍 Current path: {final_path}")
                return False

            print("✅ Repository setup verified")
            return True
        except FileNotFoundError:
            print(f"❌ Directory {REPO_DIR} not found")
            return False

    def setup_venv_manual_pip(self):
        """Create venv without pip, then install pip manually"""
        try:
            print("🔧 Creating venv without pip...")
            subprocess.run([sys.executable, "-m", "venv", ".venv", "--without-pip"],
                          check=True, timeout=60)

            print("📥 Downloading get-pip.py...")
            subprocess.run(["wget", "https://bootstrap.pypa.io/get-pip.py"], check=True)

            print("🔧 Installing pip manually...")
            subprocess.run([".venv/bin/python", "get-pip.py"], check=True)

            print("✅ Virtual environment with manual pip created successfully")
            return True
        except Exception as e:
            print(f"❌ Manual pip installation failed: {e}")
            return False

    def setup_virtual_environment(self):
        """Create virtual environment - ENFORCED for Google Colab"""
        if not self.use_venv:
            print("⚡ Lightning AI detected - using system environment (no venv)")
            return True

        print("🐍 Setting up virtual environment...")
        print(f"🔧 Platform: {self.current_platform} - venv REQUIRED")

        # Check if venv already exists
        if os.path.exists(".venv/bin/python"):
            print("✅ Virtual environment already exists and appears functional")
            return True

        print("🔨 Creating virtual environment...")

        # Try multiple methods for virtual environment creation
        venv_methods = [
            ([sys.executable, "-m", "venv", "--system-site-packages", ".venv"], "venv with system packages"),
            ([sys.executable, "-m", "venv", ".venv"], "standard venv"),
            ([sys.executable, "-m", "virtualenv", ".venv"], "virtualenv package"),
        ]

        for command, method_name in venv_methods:
            print(f"🔄 Trying: {method_name}")
            success, error_msg = self.run_command_safely(command, f"Virtual environment creation ({method_name})", timeout=60)

            if success == True:
                print("✅ Virtual environment created successfully")
                return True
            elif success == "ensurepip_failure":
                print("⚠️ ensurepip failure detected - trying manual pip installation...")
                if self.setup_venv_manual_pip():
                    return True
                break  # Don't try other methods if ensurepip failed
            else:
                print(f"❌ {method_name} failed")
                if error_msg:
                    print(f"   Error: {error_msg[:200]}")
                continue

        # Final fallback ONLY for non-Colab platforms
        if self.current_platform != "Google Colab":
            print("🔄 All virtual environment methods failed - switching to system installation")
            print("⚡ Overriding to system-wide installation")
            self.use_venv = False
            self.pip_cmd = "pip"
            self.python_cmd = "python"
            return True
        else:
            print("🚨 CRITICAL: Google Colab REQUIRES venv - cannot fallback to system")
            return False

    def install_pytorch(self):
        """Install PyTorch using enforced venv commands"""
        print("🔥 Installing PyTorch...")
        print(f"🔧 Using pip command: {self.pip_cmd}")

        pytorch_cmd = [
            self.pip_cmd, "install",
            "torch==2.6.0", "torchvision", "torchaudio",
            "--index-url", "https://download.pytorch.org/whl/test/cu124"
        ]

        success, _ = self.run_command_safely(pytorch_cmd, "PyTorch installation", timeout=300)
        return success == True

    def install_requirements(self):
        """Install requirements.txt using enforced venv commands"""
        print("📦 Installing requirements...")
        print(f"🔧 Using pip command: {self.pip_cmd}")

        if not os.path.exists("requirements.txt"):
            print("⚠️ requirements.txt not found, skipping")
            return True

        req_cmd = [self.pip_cmd, "install", "-r", "requirements.txt"]
        success, _ = self.run_command_safely(req_cmd, "Requirements installation", timeout=300)
        return success == True

    def verify_complete_setup(self):
        """Verify the complete installation using enforced venv commands"""
        print("🔍 Verifying installation...")
        print(f"🔧 Using python command: {self.python_cmd}")

        # Verify directory structure
        current_dir = os.getcwd()
        print(f"📍 Final directory: {current_dir}")

        if current_dir.count("Wan2GP") > 1:
            print("🚨 CRITICAL: Nested directory structure detected!")
            print("❌ Setup verification failed due to directory structure")
            return False

        try:
            # Test PyTorch installation
            result = subprocess.run([self.python_cmd, "-c", "import torch; print(f'PyTorch {torch.__version__} installed'); print(f'CUDA available: {torch.cuda.is_available()}')"],
                                  capture_output=True, text=True, timeout=30)

            if result.returncode == 0:
                print("✅ PyTorch verification successful:")
                for line in result.stdout.strip().split('\n'):
                    print(f"   {line}")

                # Verify main WanGP file exists
                if os.path.exists("wgp.py"):
                    print("✅ WanGP main file found")
                else:
                    print("⚠️ WanGP main file (wgp.py) not found")

                return True
            else:
                print(f"❌ PyTorch verification failed: {result.stderr}")
                return False

        except Exception as e:
            print(f"❌ Verification failed: {e}")
            return False

    def run_complete_setup(self):
        """Execute the complete setup process with enforced venv"""
        print("🚀 WAN2GP Complete System + Python Environment Setup")
        print("=" * 70)
        print(f"Platform: {self.current_platform}")
        print(f"Virtual Environment: {'Yes' if self.use_venv else 'No'}")
        print(f"Pip Command: {self.pip_cmd}")
        print(f"Python Command: {self.python_cmd}")
        print("=" * 70)

        # Final enforcement check
        if self.current_platform == "Google Colab" and not self.use_venv:
            print("🚨 FATAL: Google Colab not using venv - ABORTING")
            return False

        start_time = time.time()

        for phase_name, phase_func in self.setup_phases:
            self.update_progress(phase_name, "in_progress")

            phase_start = time.time()
            success = phase_func()
            phase_duration = time.time() - phase_start

            if success:
                self.update_progress(phase_name, "success")
                print(f"⏱️ {phase_name} completed in {phase_duration:.1f}s")
                self.current_phase += 1
            else:
                self.update_progress(phase_name, "failed")
                print(f"💥 Setup failed at: {phase_name}")
                print("\n🔄 Setup Issues - Platform-Specific Troubleshooting:")
                print("📱 Colab: Restart runtime and re-run all cells")
                print("⚡ Lightning.AI: Ensure you're using the correct Python version")
                print("🌌 Vast.AI: Check GPU drivers and CUDA installation")
                return False

        total_duration = time.time() - start_time
        print(f"\n🎉 WAN2GP setup completed successfully in {total_duration:.1f}s!")
        print("🚀 Ready to launch WAN2GP!")
        return True

# Execute the complete setup
setup = CompleteEnvironmentSetup()
setup.run_complete_setup()


✅ VERIFIED: Google Colab correctly using venv configuration
🚀 WAN2GP Complete System + Python Environment Setup
Platform: Google Colab
Virtual Environment: Yes
Pip Command: .venv/bin/pip
Python Command: .venv/bin/python

[1/7] 🔄 System Packages
Progress: 0.0% | Platform: Google Colab
🔧 Using: .venv/bin/pip | .venv/bin/python | venv: True
🔧 Installing system packages...
📦 Updating package lists...
🔧 Installing aria2, git, build-essential, and wget...
✅ System packages installed successfully

[1/7] ✅ System Packages
Progress: 0.0% | Platform: Google Colab
🔧 Using: .venv/bin/pip | .venv/bin/python | venv: True
⏱️ System Packages completed in 21.8s

[2/7] 🔄 Directory Protection
Progress: 14.3% | Platform: Google Colab
🔧 Using: .venv/bin/pip | .venv/bin/python | venv: True
🛡️ Setting up directory protection...
📍 Current directory: /content
✅ Directory protection setup complete

[2/7] ✅ Directory Protection
Progress: 14.3% | Platform: Google Colab
🔧 Using: .venv/bin/pip | .venv/bin/python | 

True

In [12]:
#@title Cell (3) Complete Model & LoRA Downloads - Fixed URLs + Permanent HF Token Storage v4.0

import os
import subprocess
import sys
import time
import json
from pathlib import Path
from IPython.display import display, HTML, Markdown
import requests
from urllib.parse import urlparse

def jupyter_lab_title(title):
    if "google.colab" not in sys.modules:
        display(Markdown(f"## {title}"))

jupyter_lab_title("Complete Model & LoRA Downloads - Fixed URLs + Permanent HF Token Storage")

# Permanent HuggingFace Token Storage System
class PermanentHFTokenManager:
    def __init__(self):
        self.token_locations = [
            "/content/.huggingface_token",  # Colab persistent location
            os.path.expanduser("~/.huggingface_token"),  # User home directory
            "/tmp/.hf_token",  # Temporary fallback
            "./.hf_token"  # Local directory
        ]
        self.token = None
        self.authenticated = False
        self.setup_token_persistence()

    def setup_token_persistence(self):
        """Setup permanent HuggingFace token storage with rich UI"""
        auth_html = f"""
        <div style="font-family: Arial, sans-serif; background: linear-gradient(135deg, #ff6b6b 0%, #ee5a24 100%);
                    color: white; padding: 15px; border-radius: 10px; margin: 10px 0;
                    box-shadow: 0 4px 15px rgba(0,0,0,0.2);">
            <h3 style="margin: 0; color: white;">🔐 Permanent HuggingFace Token Manager</h3>
            <p style="margin: 5px 0;">Store your HF token permanently for faster downloads and private model access</p>
            <div id="hf-token-status" style="margin-top: 10px;">
                <div id="token-check-result"></div>
                <button onclick="checkExistingToken()"
                        style="background: #2ed573; color: white; border: none; padding: 8px 16px;
                               border-radius: 5px; cursor: pointer; margin-right: 10px;">
                    🔍 Check Existing Token
                </button>
                <button onclick="setupNewToken()"
                        style="background: #3742fa; color: white; border: none; padding: 8px 16px;
                               border-radius: 5px; cursor: pointer; margin-right: 10px;">
                    🔑 Setup New Token
                </button>
                <button onclick="skipTokenSetup()"
                        style="background: #57606f; color: white; border: none; padding: 8px 16px;
                               border-radius: 5px; cursor: pointer;">
                    ⏭️ Skip Token Setup
                </button>
            </div>
            <div id="token-input-area" style="margin-top: 15px; display: none;">
                <input type="password" id="hf-token-input" placeholder="Enter HuggingFace Token"
                       style="padding: 8px; border-radius: 5px; border: 1px solid #ccc; margin-right: 10px; width: 300px;">
                <button onclick="saveToken()"
                        style="background: #1dd1a1; color: white; border: none; padding: 8px 16px;
                               border-radius: 5px; cursor: pointer; margin-right: 10px;">
                    💾 Save Permanently
                </button>
                <button onclick="testToken()"
                        style="background: #feca57; color: black; border: none; padding: 8px 16px;
                               border-radius: 5px; cursor: pointer;">
                    🧪 Test Token
                </button>
                <p style="font-size: 12px; margin-top: 5px;">
                    Get your token from: <a href="https://huggingface.co/settings/tokens" target="_blank"
                    style="color: #ffc048;">https://huggingface.co/settings/tokens</a>
                </p>
            </div>
        </div>
        """

        auth_js = """
        <script>
        function checkExistingToken() {
            google.colab.kernel.invokeFunction('check_existing_hf_token', [], {});
        }

        function setupNewToken() {
            document.getElementById('token-input-area').style.display = 'block';
            document.getElementById('hf-token-input').focus();
        }

        function saveToken() {
            const token = document.getElementById('hf-token-input').value;
            if (token && token.startsWith('hf_')) {
                google.colab.kernel.invokeFunction('save_hf_token_permanently', [token], {});
                document.getElementById('token-check-result').innerHTML =
                    '<p style="color: #2ed573;">💾 Saving token permanently...</p>';
            } else {
                document.getElementById('token-check-result').innerHTML =
                    '<p style="color: #ff3838;">❌ Invalid token format. Must start with "hf_"</p>';
            }
        }

        function testToken() {
            const token = document.getElementById('hf-token-input').value;
            if (token) {
                google.colab.kernel.invokeFunction('test_hf_token', [token], {});
                document.getElementById('token-check-result').innerHTML =
                    '<p style="color: #feca57;">🧪 Testing token...</p>';
            }
        }

        function skipTokenSetup() {
            google.colab.kernel.invokeFunction('skip_hf_token_setup', [], {});
            document.getElementById('hf-token-status').innerHTML =
                '<p style="color: #ffc048;">⏭️ Skipped token setup - using public access only</p>';
        }
        </script>
        """

        display(HTML(auth_html + auth_js))

    def check_existing_token(self):
        """Check for existing stored tokens"""
        for location in self.token_locations:
            try:
                if os.path.exists(location):
                    with open(location, 'r') as f:
                        potential_token = f.read().strip()
                    if potential_token.startswith('hf_') and len(potential_token) > 20:
                        self.token = potential_token
                        self.show_existing_token_found(location)
                        return True
            except Exception as e:
                continue

        self.show_no_existing_token()
        return False

    def save_token_permanently(self, token):
        """Save token to multiple persistent locations"""
        saved_locations = []

        for location in self.token_locations:
            try:
                # Create directory if needed
                os.makedirs(os.path.dirname(location) if os.path.dirname(location) else ".", exist_ok=True)

                with open(location, 'w') as f:
                    f.write(token)

                # Set restrictive permissions
                os.chmod(location, 0o600)
                saved_locations.append(location)

            except Exception as e:
                print(f"⚠️ Could not save to {location}: {e}")
                continue

        if saved_locations:
            self.token = token
            self.authenticated = True
            self.show_token_saved_success(saved_locations)
            return True
        else:
            self.show_token_save_failed()
            return False

    def test_token_validity(self, token):
        """Test if token is valid by making API call"""
        try:
            import requests
            headers = {"Authorization": f"Bearer {token}"}
            response = requests.get("https://huggingface.co/api/whoami", headers=headers, timeout=10)

            if response.status_code == 200:
                user_info = response.json()
                self.show_token_test_success(user_info.get('name', 'Unknown'))
                return True
            else:
                self.show_token_test_failed("Invalid token")
                return False

        except Exception as e:
            self.show_token_test_failed(f"Network error: {e}")
            return False

    def show_existing_token_found(self, location):
        """Show existing token found message"""
        success_html = f"""
        <div style="background: #d4edda; color: #155724; padding: 10px; border-radius: 5px;
                    border: 1px solid #c3e6cb; margin: 5px 0;">
            <strong>✅ Existing HuggingFace Token Found!</strong><br>
            📍 Location: {location}<br>
            🚀 Ready for authenticated downloads
        </div>
        """
        display(HTML(success_html))

    def show_no_existing_token(self):
        """Show no existing token message"""
        info_html = """
        <div style="background: #fff3cd; color: #856404; padding: 10px; border-radius: 5px;
                    border: 1px solid #ffeaa7; margin: 5px 0;">
            <strong>ℹ️ No Existing Token Found</strong><br>
            💡 You can setup a new token for faster downloads and private model access
        </div>
        """
        display(HTML(info_html))

    def show_token_saved_success(self, locations):
        """Show token saved successfully"""
        locations_str = "<br>".join([f"📁 {loc}" for loc in locations])
        success_html = f"""
        <div style="background: #d4edda; color: #155724; padding: 10px; border-radius: 5px;
                    border: 1px solid #c3e6cb; margin: 5px 0;">
            <strong>💾 HuggingFace Token Saved Permanently!</strong><br>
            {locations_str}<br>
            🔐 Token will persist across sessions
        </div>
        """
        display(HTML(success_html))

    def show_token_save_failed(self):
        """Show token save failed"""
        error_html = """
        <div style="background: #f8d7da; color: #721c24; padding: 10px; border-radius: 5px;
                    border: 1px solid #f5c6cb; margin: 5px 0;">
            <strong>❌ Failed to Save Token</strong><br>
            ⚠️ Continuing with public access only
        </div>
        """
        display(HTML(error_html))

    def show_token_test_success(self, username):
        """Show token test success"""
        success_html = f"""
        <div style="background: #d4edda; color: #155724; padding: 10px; border-radius: 5px;
                    border: 1px solid #c3e6cb; margin: 5px 0;">
            <strong>🧪 Token Test Successful!</strong><br>
            👤 Authenticated as: {username}<br>
            ✅ Token is valid and working
        </div>
        """
        display(HTML(success_html))

    def show_token_test_failed(self, error):
        """Show token test failed"""
        error_html = f"""
        <div style="background: #f8d7da; color: #721c24; padding: 10px; border-radius: 5px;
                    border: 1px solid #f5c6cb; margin: 5px 0;">
            <strong>🧪 Token Test Failed</strong><br>
            ❌ Error: {error}<br>
            💡 Please check your token and try again
        </div>
        """
        display(HTML(error_html))

# Enhanced Model Downloader with Fixed URLs
class FixedModelDownloader:
    def __init__(self):
        # Use global platform configuration from previous cells
        try:
            self.current_platform = current_platform
            self.pip_cmd = pip_cmd
            self.python_cmd = python_cmd
            self.use_venv = use_venv
        except NameError:
            print("⚠️ Platform detection variables not found. Using fallback detection.")
            self.detect_platform_fallback()

        # CRITICAL ENFORCEMENT: Ensure venv for Google Colab
        self.enforce_colab_venv()

        # Initialize permanent HF token manager
        self.hf_token_manager = PermanentHFTokenManager()

        # Fixed repository URL with correct naming
        self.base_repo_url = "https://huggingface.co/Kijai/WanVideo_comfy"
        self.resolve_url = f"{self.base_repo_url}/resolve/main"

        # Download methods with HF Hub integration
        self.download_methods = [
            ("hf_hub", self.download_with_hf_hub),
            ("aria2c", self.download_with_aria2c),
            ("wget", self.download_with_wget),
            ("python", self.download_with_python)
        ]

        # CORRECTED model definitions based on actual repository files
        self.setup_corrected_model_definitions()

        # Download statistics
        self.download_stats = {
            "total_files": 0,
            "downloaded_files": 0,
            "failed_files": 0,
            "total_size": 0,
            "downloaded_size": 0
        }

    def enforce_colab_venv(self):
        """CRITICAL: Enforce venv usage for Google Colab"""
        if hasattr(self, 'current_platform') and self.current_platform == "Google Colab":
            if not hasattr(self, 'use_venv') or not self.use_venv or self.pip_cmd != ".venv/bin/pip":
                print("🚨 ENFORCING: Google Colab MUST use venv for downloads")
                self.use_venv = True
                self.pip_cmd = ".venv/bin/pip"
                self.python_cmd = ".venv/bin/python"
                print("✅ ENFORCED: Download system using venv commands")
            else:
                print("✅ VERIFIED: Download system correctly using venv for Colab")

    def detect_platform_fallback(self):
        """Fallback platform detection - ENFORCES VENV FOR COLAB"""
        if "google.colab" in sys.modules:
            self.current_platform = "Google Colab"
            # ENFORCED: Always venv for Colab
            self.pip_cmd, self.python_cmd, self.use_venv = ".venv/bin/pip", ".venv/bin/python", True
        elif any(["lightning" in str(sys.executable).lower(), "teamspace" in os.getcwd()]):
            self.current_platform = "Lightning AI"
            self.pip_cmd, self.python_cmd, self.use_venv = "pip", "python", False
        else:
            self.current_platform = "Generic"
            self.pip_cmd, self.python_cmd, self.use_venv = ".venv/bin/pip", ".venv/bin/python", True

    def setup_corrected_model_definitions(self):
        """Setup model definitions with CORRECTED URLs matching actual repository files"""

        # Core Models - Based on actual repository structure
        self.models = {
            "wan_14b_phantom": {
                "url": f"{self.resolve_url}/Phantom-Wan-14B_fp16.safetensors",
                "filename": "Phantom-Wan-14B_fp16.safetensors",
                "size": "29.1 GB",
                "path": "ckpts/",
                "description": "Phantom Wan 14B model - Best for object/person transfer"
            },
            "wan_14b_skyreels": {
                "url": f"{self.resolve_url}/SkyReels-v2-14B_fp16.safetensors",
                "filename": "SkyReels-v2-14B_fp16.safetensors",
                "size": "29.1 GB",
                "path": "ckpts/",
                "description": "SkyReels v2 14B - High quality video generation"
            },
            "wan_1_3b_base": {
                "url": f"{self.resolve_url}/Wan2_1-T2V-1_3B_bf16.safetensors",
                "filename": "Wan2_1-T2V-1_3B_bf16.safetensors",
                "size": "2.87 GB",
                "path": "ckpts/",
                "description": "Wan 2.1 1.3B model - Fast generation for lower hardware"
            },
            "wan_1_3b_fp32": {
                "url": f"{self.resolve_url}/Wan2_1-T2V-1_3B_fp32.safetensors",
                "filename": "Wan2_1-T2V-1_3B_fp32.safetensors",
                "size": "5.68 GB",
                "path": "ckpts/",
                "description": "Wan 2.1 1.3B FP32 - Higher precision variant"
            }
        }

        # Text Encoders
        self.text_encoders = {
            "umt5_xxl_fp16": {
                "url": f"{self.resolve_url}/umt5_xxl_fp16.safetensors",
                "filename": "umt5_xxl_fp16.safetensors",
                "size": "2.98 GB",
                "path": "text_encoders/",
                "description": "T5 text encoder - Required for all video generation"
            }
        }

        # VAE Models
        self.vaes = {
            "wan_vae_bf16": {
                "url": f"{self.resolve_url}/Wan2_1_VAE_bf16.safetensors",
                "filename": "Wan2_1_VAE_bf16.safetensors",
                "size": "254 MB",
                "path": "vae/",
                "description": "Video VAE - Converts between pixel and latent space"
            },
            "wan_vae_fp32": {
                "url": f"{self.resolve_url}/Wan2_1_VAE_fp32.safetensors",
                "filename": "Wan2_1_VAE_fp32.safetensors",
                "size": "508 MB",
                "path": "vae/",
                "description": "Video VAE FP32 - Higher precision variant"
            }
        }

        # CLIP Vision
        self.clip_vision = {
            "clip_vision_h": {
                "url": f"{self.resolve_url}/clip_vision_h.safetensors",
                "filename": "clip_vision_h.safetensors",
                "size": "1.47 GB",
                "path": "clip_vision/",
                "description": "CLIP vision encoder for image-to-video tasks"
            }
        }

        # LoRAs - Using corrected URLs from documentation
        self.loras = {
            "causVid_14b": {
                "url": f"{self.resolve_url}/Wan21_CausVid_14B_T2V_lora_rank32.safetensors",
                "filename": "Wan21_CausVid_14B_T2V_lora_rank32.safetensors",
                "size": "323 MB",
                "path": "loras14B/",
                "description": "CausVid LoRA - 4-12 step generation with 2x speed improvement"
            },
            "accVid_t2v_14b": {
                "url": f"{self.resolve_url}/Wan21AccVidT2V14Blorarank32fp16.safetensors",
                "filename": "Wan21AccVidT2V14Blorarank32fp16.safetensors",
                "size": "317 MB",
                "path": "loras14B/",
                "description": "AccVid T2V LoRA - 2x speed with no classifier-free guidance"
            },
            "accVid_i2v_14b": {
                "url": f"{self.resolve_url}/Wan21AccVidI2V480P14Blorarank32fp16.safetensors",
                "filename": "Wan21AccVidI2V480P14Blorarank32fp16.safetensors",
                "size": "319 MB",
                "path": "lorasi2v/",
                "description": "AccVid I2V LoRA - Image-to-video acceleration"
            },
            "safe_forcing_14b": {
                "url": f"{self.resolve_url}/Wan21T2V14Blightx2vcfgstepdistilllorarank32.safetensors",
                "filename": "Wan21T2V14Blightx2vcfgstepdistilllorarank32.safetensors",
                "size": "311 MB",
                "path": "loras14B/",
                "description": "Safe-Forcing LoRA - 2-8 step generation with 2x speed"
            }
        }

    def create_directory_structure(self):
        """Create necessary directories for downloads"""
        directories = [
            "ckpts", "text_encoders", "vae", "vae_approx", "clip_vision",
            "loras", "loras1.3B", "loras14B", "lorasi2v", "lorashunyuan", "lorasltxv"
        ]

        print("📁 Creating directory structure...")
        for directory in directories:
            Path(directory).mkdir(parents=True, exist_ok=True)
            print(f"   ✅ {directory}/")

        print("✅ Directory structure created successfully")

    def download_with_hf_hub(self, url, filename, output_dir):
        """Download using HuggingFace Hub API with authentication"""
        try:
            # Try installing huggingface_hub if not available
            try:
                from huggingface_hub import hf_hub_download
            except ImportError:
                print("📦 Installing huggingface_hub...")
                subprocess.run([self.pip_cmd, "install", "huggingface_hub"], check=True)
                from huggingface_hub import hf_hub_download

            # Parse HuggingFace URL to extract repo and filename
            if "/resolve/main/" in url:
                parts = url.split("/resolve/main/")
                repo_path = parts[0].replace("https://huggingface.co/", "")
                file_path = parts[1]
            else:
                print("❌ Invalid HuggingFace URL format")
                return False

            print(f"📥 Downloading with HuggingFace Hub API...")
            print(f"📁 Repo: {repo_path}")
            print(f"📄 File: {file_path}")

            # Use token if available
            token = self.hf_token_manager.token if self.hf_token_manager.authenticated else None

            downloaded_file = hf_hub_download(
                repo_id=repo_path,
                filename=file_path,
                token=token,
                local_dir=output_dir,
                local_dir_use_symlinks=False,
                resume_download=True
            )

            print(f"✅ HuggingFace Hub download successful")
            return True

        except Exception as e:
            print(f"❌ HuggingFace Hub download failed: {e}")
            return False

    def download_with_aria2c(self, url, filename, output_dir):
        """Download using aria2c with authentication headers"""
        try:
            output_path = os.path.join(output_dir, filename)

            cmd = [
                "aria2c",
                "-x", "16",        # 16 connections
                "-s", "16",        # 16 segments
                "-k", "1M",        # 1MB per segment
                "--continue",      # Resume capability
                "--dir", output_dir,
                "--out", filename,
                url
            ]

            # Add authentication headers if available
            if self.hf_token_manager.authenticated:
                cmd.extend(["--header", f"Authorization: Bearer {self.hf_token_manager.token}"])

            print(f"📥 Downloading with aria2c (16 connections)...")
            result = subprocess.run(cmd, capture_output=True, text=True, timeout=1800)

            if result.returncode == 0:
                print(f"✅ aria2c download successful")
                return True
            else:
                print(f"❌ aria2c failed: {result.stderr[:200]}")
                return False

        except subprocess.TimeoutExpired:
            print(f"⏰ aria2c download timed out")
            return False
        except Exception as e:
            print(f"❌ aria2c error: {e}")
            return False

    def download_with_wget(self, url, filename, output_dir):
        """Download using wget with authentication"""
        try:
            output_path = os.path.join(output_dir, filename)

            cmd = [
                "wget",
                "--continue",              # Resume capability
                "--progress=bar",          # Progress bar
                "--timeout=30",            # Connection timeout
                "--tries=3",               # Retry attempts
                "-O", output_path,
                url
            ]

            # Add authentication headers if available
            if self.hf_token_manager.authenticated:
                cmd.extend(["--header", f"Authorization: Bearer {self.hf_token_manager.token}"])

            print(f"📥 Downloading with wget...")
            result = subprocess.run(cmd, capture_output=True, text=True, timeout=1800)

            if result.returncode == 0:
                print(f"✅ wget download successful")
                return True
            else:
                print(f"❌ wget failed: {result.stderr[:200]}")
                return False

        except subprocess.TimeoutExpired:
            print(f"⏰ wget download timed out")
            return False
        except Exception as e:
            print(f"❌ wget error: {e}")
            return False

    def download_with_python(self, url, filename, output_dir):
        """Download using Python urllib with authentication"""
        try:
            import urllib.request
            import shutil

            output_path = os.path.join(output_dir, filename)

            print(f"📥 Downloading with Python urllib...")

            # Create request with authentication if available
            req = urllib.request.Request(url)
            if self.hf_token_manager.authenticated:
                req.add_header('Authorization', f'Bearer {self.hf_token_manager.token}')

            with urllib.request.urlopen(req) as response:
                with open(output_path, 'wb') as out_file:
                    shutil.copyfileobj(response, out_file)

            print(f"✅ Python download successful")
            return True

        except Exception as e:
            print(f"❌ Python download failed: {e}")
            return False

    def download_file(self, url, filename, output_dir, size_info="Unknown"):
        """Download a single file with progressive fallback methods"""
        output_path = os.path.join(output_dir, filename)

        # Check if file already exists
        if os.path.exists(output_path):
            print(f"📄 {filename} already exists - skipping")
            return True

        print(f"\n📥 Downloading: {filename}")
        print(f"📊 Size: {size_info}")
        print(f"🔗 URL: {url}")
        print(f"📁 Destination: {output_path}")

        # Try each download method in order
        for method_name, method_func in self.download_methods:
            print(f"\n🔄 Trying {method_name}...")

            try:
                if method_func(url, filename, output_dir):
                    print(f"✅ Download successful with {method_name}")
                    return True
                else:
                    print(f"❌ {method_name} failed - trying next method")
                    continue

            except Exception as e:
                print(f"❌ {method_name} crashed: {e}")
                continue

        print(f"💥 All download methods failed for {filename}")
        return False

    def download_category(self, category_name, items_dict):
        """Download all items in a category"""
        print(f"\n🎯 Downloading {category_name}...")
        print("=" * 60)

        success_count = 0
        total_count = len(items_dict)

        for item_key, item_info in items_dict.items():
            print(f"\n[{success_count + 1}/{total_count}] Processing {item_key}...")

            if self.download_file(
                item_info["url"],
                item_info["filename"],
                item_info["path"],
                item_info["size"]
            ):
                success_count += 1
                self.download_stats["downloaded_files"] += 1
            else:
                self.download_stats["failed_files"] += 1

            self.download_stats["total_files"] += 1

        print(f"\n✅ {category_name} Downloads Complete: {success_count}/{total_count} successful")
        return success_count == total_count

    def run_complete_download_system(self):
        """Run the complete download system with corrected URLs and permanent token storage"""
        print("🚀 WAN2GP Model and LoRA Download System")
        print("=" * 70)
        print(f"Platform: {self.current_platform}")
        print(f"Virtual Environment: {'Yes' if self.use_venv else 'No'}")
        print(f"Pip Command: {self.pip_cmd}")
        print(f"Python Command: {self.python_cmd}")
        print(f"Download Methods: HF Hub → aria2c → wget → python urllib")
        print(f"Repository: {self.base_repo_url}")
        print("=" * 70)

        # Final enforcement check
        if hasattr(self, 'current_platform') and self.current_platform == "Google Colab" and not self.use_venv:
            print("🚨 FATAL: Google Colab not using venv - ABORTING DOWNLOADS")
            return False

        # Initialize token system
        print("\n🔐 Checking for permanent HuggingFace token...")
        self.hf_token_manager.check_existing_token()

        # Create directory structure
        self.create_directory_structure()

        # Download categories with corrected model definitions
        download_categories = [
            ("Essential Models", self.models),
            ("Text Encoders", self.text_encoders),
            ("VAE Models", self.vaes),
            ("CLIP Vision", self.clip_vision),
            ("Performance LoRAs", self.loras)
        ]

        print("\n🎯 Available Download Categories (CORRECTED URLs):")
        for i, (category_name, items) in enumerate(download_categories, 1):
            item_count = len(items)
            total_size = 0
            for item in items.values():
                try:
                    size_parts = item["size"].split()
                    if len(size_parts) >= 2:
                        number = float(size_parts[0])
                        unit = size_parts[1].upper()
                        multipliers = {"GB": 1, "MB": 0.001}
                        total_size += number * multipliers.get(unit, 0)
                except:
                    continue
            print(f"{i}. {category_name} ({item_count} files, ~{total_size:.1f} GB)")

        print("\n📋 Download Options:")
        print("A. Download All (Recommended)")
        print("B. Download Essential Only (Models + Text Encoders + VAE)")
        print("C. Download LoRAs Only")
        print("D. Download Models Only")

        choice = input("\n🎯 Select download option (A/B/C/D): ").strip().upper()

        if choice == "A":
            print("\n🚀 Downloading All Categories...")
            for category_name, items in download_categories:
                self.download_category(category_name, items)

        elif choice == "B":
            print("\n🚀 Downloading Essential Components...")
            essential_categories = [
                ("Essential Models", self.models),
                ("Text Encoders", self.text_encoders),
                ("VAE Models", self.vaes)
            ]
            for category_name, items in essential_categories:
                self.download_category(category_name, items)

        elif choice == "C":
            print("\n🚀 Downloading LoRAs Only...")
            self.download_category("Performance LoRAs", self.loras)

        elif choice == "D":
            print("\n🚀 Downloading Models Only...")
            self.download_category("Essential Models", self.models)

        else:
            print("❌ Invalid choice. Defaulting to Essential Only.")
            essential_categories = [
                ("Essential Models", self.models),
                ("Text Encoders", self.text_encoders),
                ("VAE Models", self.vaes)
            ]
            for category_name, items in essential_categories:
                self.download_category(category_name, items)

        # Final statistics
        print("\n📊 Download Statistics:")
        print(f"Total files processed: {self.download_stats['total_files']}")
        print(f"Successfully downloaded: {self.download_stats['downloaded_files']}")
        print(f"Failed downloads: {self.download_stats['failed_files']}")

        success_rate = (self.download_stats['downloaded_files'] / max(self.download_stats['total_files'], 1)) * 100
        print(f"Success rate: {success_rate:.1f}%")

        if self.download_stats['failed_files'] == 0:
            print("\n🎉 All downloads completed successfully!")
            print("✅ Ready to launch WAN2GP!")
        else:
            print(f"\n⚠️ {self.download_stats['failed_files']} files failed to download")
            print("🔄 You can re-run this cell to retry failed downloads")

        return self.download_stats['failed_files'] == 0

# Colab function bindings for JavaScript integration
def check_existing_hf_token():
    """Check for existing HF token (called from JavaScript)"""
    global downloader
    downloader.hf_token_manager.check_existing_token()

def save_hf_token_permanently(token):
    """Save HF token permanently (called from JavaScript)"""
    global downloader
    if downloader.hf_token_manager.save_token_permanently(token):
        downloader.hf_token_manager.authenticated = True
        print("🔑 HuggingFace token saved and authenticated successfully!")
    else:
        print("❌ Failed to save HuggingFace token")

def test_hf_token(token):
    """Test HF token validity (called from JavaScript)"""
    global downloader
    if downloader.hf_token_manager.test_token_validity(token):
        print("🧪 Token test successful - ready for authenticated downloads!")
    else:
        print("❌ Token test failed - please check your token")

def skip_hf_token_setup():
    """Skip HF token setup (called from JavaScript)"""
    print("⏭️ Skipped HuggingFace token setup - using public access only")

# Execute the enhanced download system with fixed URLs and permanent token storage
downloader = FixedModelDownloader()
downloader.run_complete_download_system()


✅ VERIFIED: Download system correctly using venv for Colab


🚀 WAN2GP Model and LoRA Download System
Platform: Google Colab
Virtual Environment: Yes
Pip Command: .venv/bin/pip
Python Command: .venv/bin/python
Download Methods: HF Hub → aria2c → wget → python urllib
Repository: https://huggingface.co/Kijai/WanVideo_comfy

🔐 Checking for permanent HuggingFace token...


📁 Creating directory structure...
   ✅ ckpts/
   ✅ text_encoders/
   ✅ vae/
   ✅ vae_approx/
   ✅ clip_vision/
   ✅ loras/
   ✅ loras1.3B/
   ✅ loras14B/
   ✅ lorasi2v/
   ✅ lorashunyuan/
   ✅ lorasltxv/
✅ Directory structure created successfully

🎯 Available Download Categories (CORRECTED URLs):
1. Essential Models (4 files, ~66.8 GB)
2. Text Encoders (1 files, ~3.0 GB)
3. VAE Models (2 files, ~0.8 GB)
4. CLIP Vision (1 files, ~1.5 GB)
5. Performance LoRAs (4 files, ~1.3 GB)

📋 Download Options:
A. Download All (Recommended)
B. Download Essential Only (Models + Text Encoders + VAE)
C. Download LoRAs Only
D. Download Models Only


KeyboardInterrupt: Interrupted by user

In [13]:
#@title Cell (4) WAN2GP Launch - Enhanced with venv Enforcement v3.2

import os
import subprocess
import sys

def jupyter_lab_title(title):
    if "google.colab" not in sys.modules:
        from IPython.display import display, Markdown
        display(Markdown(f"## {title}"))

jupyter_lab_title("WAN2GP Launch - Enhanced with venv Enforcement")

class WAN2GPLauncher:
    def __init__(self):
        # Use global platform configuration from previous cells
        try:
            self.current_platform = current_platform
            self.pip_cmd = pip_cmd
            self.python_cmd = python_cmd
            self.use_venv = use_venv
        except NameError:
            print("⚠️ Platform detection variables not found. Using fallback detection.")
            self.detect_platform_fallback()

        # CRITICAL ENFORCEMENT: Ensure venv for Google Colab
        self.enforce_colab_venv()

    def enforce_colab_venv(self):
        """CRITICAL: Enforce venv usage for Google Colab"""
        if self.current_platform == "Google Colab":
            if not self.use_venv or self.python_cmd != ".venv/bin/python":
                print("🚨 ENFORCING: Google Colab MUST use venv for WAN2GP launch")
                self.use_venv = True
                self.pip_cmd = ".venv/bin/pip"
                self.python_cmd = ".venv/bin/python"
                print("✅ ENFORCED: Launch system using venv python command")
            else:
                print("✅ VERIFIED: Launch system correctly using venv for Colab")

    def detect_platform_fallback(self):
        """Fallback platform detection - ENFORCES VENV FOR COLAB"""
        if "google.colab" in sys.modules:
            self.current_platform = "Google Colab"
            # ENFORCED: Always venv for Colab
            self.pip_cmd, self.python_cmd, self.use_venv = ".venv/bin/pip", ".venv/bin/python", True
        elif any(["lightning" in str(sys.executable).lower(), "teamspace" in os.getcwd()]):
            self.current_platform = "Lightning AI"
            self.pip_cmd, self.python_cmd, self.use_venv = "pip", "python", False
        else:
            self.current_platform = "Generic"
            self.pip_cmd, self.python_cmd, self.use_venv = ".venv/bin/pip", ".venv/bin/python", True

    def launch_wangp(self, model_type="t2v-1-3B", additional_args=None):
        """Launch WAN2GP with enforced venv python command"""
        print("🚀 Launching WAN2GP...")
        print("=" * 50)
        print(f"Platform: {self.current_platform}")
        print(f"Python Command: {self.python_cmd}")
        print(f"Model Type: {model_type}")
        print("=" * 50)

        # Final enforcement check
        if self.current_platform == "Google Colab" and not self.use_venv:
            print("🚨 FATAL: Google Colab not using venv - ABORTING LAUNCH")
            return False

        # Verify we're in the correct directory
        if not os.path.exists("wgp.py"):
            print("❌ wgp.py not found. Make sure you're in the Wan2GP directory.")
            print(f"📍 Current directory: {os.getcwd()}")
            return False

        # Build launch command
        launch_cmd = [self.python_cmd, "wgp.py --share", f"--{model_type}"]

        # Add additional arguments if provided
        if additional_args:
            launch_cmd.extend(additional_args)

        print(f"🎯 Launch command: {' '.join(launch_cmd)}")
        print("\n🔄 Starting WAN2GP...")
        print("🌐 Once launched, access the interface at: http://localhost:7860")
        print("⏹️ Press Ctrl+C to stop WAN2GP")

        try:
            # Launch WAN2GP
            subprocess.run(launch_cmd)
        except KeyboardInterrupt:
            print("\n⏹️ WAN2GP stopped by user")
        except Exception as e:
            print(f"❌ Launch failed: {e}")
            return False

        return True

    def show_launch_options(self):
        """Show available launch options"""
        print("🎯 WAN2GP Launch Options:")
        print("=" * 50)

        options = {
            "1": ("t2v-1-3B", "Wan 2.1 Text2Video 1.3B (Fast, 6GB VRAM)"),
            "2": ("t2v-14B", "Wan 2.1 Text2Video 14B (High Quality, 12GB VRAM)"),
            "3": ("i2v-1-3B", "Wan Fun InP 1.3B (Image2Video, 6GB VRAM)"),
            "4": ("i2v-14B", "Wan 2.1 Image2Video 14B (High Quality I2V, 12GB VRAM)"),
            "5": ("vace-1-3B", "Wan Vace 1.3B (ControlNet, 6GB VRAM)"),
            "6": ("vace-14B", "Wan Vace 14B (ControlNet High Quality, 12GB VRAM)")
        }

        for key, (model, description) in options.items():
            print(f"{key}. {description}")

        print("\n🔧 Performance Options:")
        print("A. Low VRAM Mode (--profile 4 --attention sdpa)")
        print("B. High Performance Mode (--compile --attention sage)")
        print("C. Custom Launch (enter your own arguments)")

        choice = input("\n🎯 Select option (1-6, A-C): ").strip()

        additional_args = []

        if choice in options:
            model_type = options[choice][0]
        elif choice.upper() == "A":
            model_type = "t2v-1-3B"
            additional_args = ["--profile", "4", "--attention", "sdpa"]
        elif choice.upper() == "B":
            model_type = "t2v-14B"
            additional_args = ["--compile", "--attention", "sage"]
        elif choice.upper() == "C":
            model_type = input("Enter model type (e.g., t2v-1-3B): ").strip()
            custom_args = input("Enter additional arguments (optional): ").strip()
            if custom_args:
                additional_args = custom_args.split()
        else:
            print("❌ Invalid choice. Using default t2v-1-3B")
            model_type = "t2v-1-3B"

        return model_type, additional_args

# Create launcher instance
launcher = WAN2GPLauncher()

# Show options and launch
model_type, additional_args = launcher.show_launch_options()
launcher.launch_wangp(model_type, additional_args)


✅ VERIFIED: Launch system correctly using venv for Colab
🎯 WAN2GP Launch Options:
1. Wan 2.1 Text2Video 1.3B (Fast, 6GB VRAM)
2. Wan 2.1 Text2Video 14B (High Quality, 12GB VRAM)
3. Wan Fun InP 1.3B (Image2Video, 6GB VRAM)
4. Wan 2.1 Image2Video 14B (High Quality I2V, 12GB VRAM)
5. Wan Vace 1.3B (ControlNet, 6GB VRAM)
6. Wan Vace 14B (ControlNet High Quality, 12GB VRAM)

🔧 Performance Options:
A. Low VRAM Mode (--profile 4 --attention sdpa)
B. High Performance Mode (--compile --attention sage)
C. Custom Launch (enter your own arguments)

🎯 Select option (1-6, A-C): 1
🚀 Launching WAN2GP...
Platform: Google Colab
Python Command: .venv/bin/python
Model Type: t2v-1-3B
🎯 Launch command: .venv/bin/python wgp.py --share --t2v-1-3B

🔄 Starting WAN2GP...
🌐 Once launched, access the interface at: http://localhost:7860
⏹️ Press Ctrl+C to stop WAN2GP


True

In [14]:
!.venv/bin/python wgp.py --share --t2v-1-3B

Traceback (most recent call last):
  File "/content/Wan2GP/wgp.py", line 16, in <module>
    import wan
  File "/content/Wan2GP/wan/__init__.py", line 3, in <module>
    from .text2video import WanT2V
  File "/content/Wan2GP/wan/text2video.py", line 31, in <module>
    from wgp import update_loras_slists
  File "/content/Wan2GP/wgp.py", line 41, in <module>
    from preprocessing.matanyone  import app as matanyone_app
  File "/content/Wan2GP/preprocessing/matanyone/app.py", line 16, in <module>
    from .tools.interact_tools import SamControler
  File "/content/Wan2GP/preprocessing/matanyone/tools/interact_tools.py", line 8, in <module>
    import matplotlib.pyplot as plt
  File "/content/Wan2GP/.venv/lib/python3.11/site-packages/matplotlib/__init__.py", line 1296, in <module>
    rcParams['backend'] = os.environ.get('MPLBACKEND')
    ~~~~~~~~^^^^^^^^^^^
  File "/content/Wan2GP/.venv/lib/python3.11/site-packages/matplotlib/__init__.py", line 771, in __setitem__
    raise ValueError(f"K

In [5]:
#@title Cell (1) - WAN2GP Comprehensive Diagnostic and Auto-Repair System (Latest v6.2 Compatible)

import sys
import os
import subprocess
import shutil
import time
import json
from pathlib import Path
from IPython.display import display, HTML, Markdown
import importlib.util

class WAN2GPDiagnosticAndRepair:
    def __init__(self):
        self.platform = self.detect_platform()
        self.pip_cmd, self.python_cmd, self.use_venv = self.get_platform_commands()
        self.issues_found = []
        self.repairs_applied = []
        self.gpu_info = {}

    def detect_platform(self):
        """Enhanced platform detection with comprehensive indicators"""
        # Lightning AI detection - multiple indicators for reliability
        lightning_indicators = [
            "lightning" in str(sys.executable).lower(),
            "teamspace-studios" in os.getcwd(),
            "LIGHTNING_CLOUDSPACE_HOST" in os.environ,
            "LIGHTNING_CLOUDSPACE_ID" in os.environ,
            "/commands/python" in str(sys.executable),
            "/home/zeus/miniconda3/envs/cloudspace" in str(sys.executable),
            os.path.exists("/teamspace"),
            os.path.exists("/commands")
        ]

        # Google Colab detection
        colab_indicators = [
            "google.colab" in sys.modules,
            "/content" in os.getcwd()
        ]

        # Vast.AI detection
        vast_indicators = [
            "VAST_CONTAINER_LABEL" in os.environ,
            "/workspace" in os.getcwd(),
            "vast" in os.environ.get("HOSTNAME", "").lower()
        ]

        if any(lightning_indicators):
            return "Lightning AI"
        elif any(colab_indicators):
            return "Google Colab"
        elif any(vast_indicators):
            return "Vast.AI/Generic"
        else:
            return "Vast.AI/Generic"

    def get_platform_commands(self):
        """Get platform-specific pip and python commands"""
        if self.platform == "Lightning AI":
            return "pip", "python", False  # (pip_cmd, python_cmd, use_venv)
        elif self.platform == "Google Colab":
            return ".venv/bin/pip", ".venv/bin/python", True
        else:  # Vast.AI/Generic
            return ".venv/bin/pip", ".venv/bin/python", True

    def run_command_safely(self, command, description, timeout=60):
        """Execute command with comprehensive error handling"""
        try:
            result = subprocess.run(
                command, capture_output=True, text=True, timeout=timeout, shell=isinstance(command, str)
            )
            if result.returncode == 0:
                return True, result.stdout
            else:
                return False, result.stderr
        except subprocess.TimeoutExpired:
            return False, f"Timeout after {timeout}s"
        except Exception as e:
            return False, str(e)

    def check_gpu_compatibility(self):
        """Comprehensive GPU detection and compatibility check"""
        print("🔍 Checking GPU compatibility...")

        # Check NVIDIA GPU presence
        success, output = self.run_command_safely("nvidia-smi", "GPU Detection")
        if not success:
            self.issues_found.append("No NVIDIA GPU detected or nvidia-smi not available")
            return False

        # Parse GPU info
        try:
            # Extract GPU name and VRAM from nvidia-smi output
            lines = output.split('\n')
            for line in lines:
                if 'RTX' in line or 'GTX' in line or 'Tesla' in line or 'A100' in line:
                    gpu_name = line.split('|')[1].strip() if '|' in line else "Unknown GPU"
                    self.gpu_info['name'] = gpu_name
                    break

            # Check VRAM
            for line in lines:
                if 'MiB' in line and '/' in line:
                    vram_info = [part for part in line.split() if 'MiB' in part]
                    if len(vram_info) >= 2:
                        total_vram = int(vram_info[-1].replace('MiB', ''))
                        self.gpu_info['vram_mb'] = total_vram
                        self.gpu_info['vram_gb'] = total_vram / 1024
                        break

            print(f"✅ GPU detected: {self.gpu_info.get('name', 'Unknown')}")
            print(f"✅ VRAM: {self.gpu_info.get('vram_gb', 0):.1f}GB")

            # Check VRAM adequacy
            vram_gb = self.gpu_info.get('vram_gb', 0)
            if vram_gb < 6:
                self.issues_found.append(f"Low VRAM detected ({vram_gb:.1f}GB). Minimum 6GB recommended.")

            return True

        except Exception as e:
            self.issues_found.append(f"GPU info parsing failed: {str(e)}")
            return False

    def check_cuda_pytorch_compatibility(self):
        """Check CUDA and PyTorch compatibility"""
        print("🔍 Checking CUDA and PyTorch compatibility...")

        # Check CUDA version
        success, cuda_output = self.run_command_safely("nvcc --version", "CUDA Version Check")
        if not success:
            success, cuda_output = self.run_command_safely("nvidia-smi", "CUDA Runtime Check")

        # Check PyTorch installation
        try:
            import torch
            pytorch_version = torch.__version__
            cuda_available = torch.cuda.is_available()

            print(f"✅ PyTorch version: {pytorch_version}")
            print(f"✅ CUDA available in PyTorch: {cuda_available}")

            if not cuda_available:
                self.issues_found.append("PyTorch cannot detect CUDA")
                return False

            # Check for version compatibility
            if "50" in self.gpu_info.get('name', '') and not pytorch_version.startswith('2.7'):
                self.issues_found.append("RTX 50XX series requires PyTorch 2.7.0 or newer")

            return True

        except ImportError:
            self.issues_found.append("PyTorch not installed")
            return False
        except Exception as e:
            self.issues_found.append(f"PyTorch check failed: {str(e)}")
            return False

    def check_python_version(self):
        """Check Python version compatibility"""
        print("🔍 Checking Python version...")

        python_version = sys.version_info
        version_string = f"{python_version.major}.{python_version.minor}.{python_version.micro}"
        print(f"✅ Python version: {version_string}")

        if python_version.major != 3 or python_version.minor != 10:
            self.issues_found.append(f"Python {version_string} detected. Python 3.10.9 recommended for best compatibility.")

        return True

    def check_wan2gp_installation(self):
        """Check WAN2GP installation and repository"""
        print("🔍 Checking WAN2GP installation...")

        wan2gp_paths = ["Wan2GP", "wan2gp", "WAN2GP", "../Wan2GP", "../wan2gp"]
        wan2gp_found = False

        for path in wan2gp_paths:
            if os.path.exists(path):
                wan2gp_found = True
                wgp_py_path = os.path.join(path, "wgp.py")
                if os.path.exists(wgp_py_path):
                    print(f"✅ WAN2GP found at: {path}")
                    return True
                break

        if not wan2gp_found:
            self.issues_found.append("WAN2GP repository not found")
            return False

        self.issues_found.append("WAN2GP repository found but wgp.py missing")
        return False

    def check_dependencies(self):
        """Check critical dependencies"""
        print("🔍 Checking critical dependencies...")

        critical_deps = {
            'torch': 'PyTorch',
            'torchvision': 'TorchVision',
            'gradio': 'Gradio',
            'transformers': 'Transformers',
            'accelerate': 'Accelerate',
            'diffusers': 'Diffusers'
        }

        missing_deps = []
        for dep, name in critical_deps.items():
            try:
                importlib.import_module(dep)
                print(f"✅ {name} installed")
            except ImportError:
                missing_deps.append(dep)
                print(f"❌ {name} missing")

        if missing_deps:
            self.issues_found.append(f"Missing dependencies: {', '.join(missing_deps)}")
            return False

        return True

    def check_attention_mechanisms(self):
        """Check available attention mechanisms"""
        print("🔍 Checking attention mechanisms...")

        attention_status = {}

        # Check Triton
        try:
            import triton
            attention_status['triton'] = f"✅ Triton {triton.__version__}"
        except ImportError:
            attention_status['triton'] = "❌ Triton not available"

        # Check SageAttention
        try:
            import sageattention
            attention_status['sage'] = "✅ SageAttention available"
        except ImportError:
            attention_status['sage'] = "❌ SageAttention not available"

        # Check Flash Attention
        try:
            import flash_attn
            attention_status['flash'] = "✅ Flash Attention available"
        except ImportError:
            attention_status['flash'] = "❌ Flash Attention not available"

        for mech, status in attention_status.items():
            print(f"  {status}")

        # Recommend fallback if advanced attention not available
        if all("❌" in status for status in attention_status.values()):
            self.issues_found.append("No advanced attention mechanisms available. Will use SDPA fallback.")

        return True

    def check_memory_and_performance(self):
        """Check memory and performance configuration"""
        print("🔍 Checking memory and performance...")

        # Check available system RAM
        try:
            import psutil
            ram_gb = psutil.virtual_memory().total / (1024**3)
            print(f"✅ System RAM: {ram_gb:.1f}GB")

            if ram_gb < 16:
                self.issues_found.append(f"Low system RAM ({ram_gb:.1f}GB). 16GB+ recommended for best performance.")
        except ImportError:
            print("⚠️ Cannot check system RAM (psutil not available)")

        # Check disk space
        try:
            disk_usage = shutil.disk_usage(os.getcwd())
            free_gb = disk_usage.free / (1024**3)
            print(f"✅ Free disk space: {free_gb:.1f}GB")

            if free_gb < 20:
                self.issues_found.append(f"Low disk space ({free_gb:.1f}GB). 20GB+ recommended.")
        except Exception:
            print("⚠️ Cannot check disk space")

        return True

    def auto_repair_pytorch(self):
        """Auto-repair PyTorch installation"""
        print("🔧 Attempting PyTorch repair...")

        # Determine correct PyTorch version based on GPU
        if "50" in self.gpu_info.get('name', ''):
            # RTX 50XX series
            pytorch_cmd = [
                self.pip_cmd, "install", "--upgrade",
                "torch==2.7.0", "torchvision", "torchaudio",
                "--index-url", "https://download.pytorch.org/whl/test/cu128"
            ]
        else:
            # RTX 10XX-40XX series
            pytorch_cmd = [
                self.pip_cmd, "install", "--upgrade",
                "torch==2.6.0", "torchvision", "torchaudio",
                "--index-url", "https://download.pytorch.org/whl/test/cu124"
            ]

        success, output = self.run_command_safely(pytorch_cmd, "PyTorch Installation", timeout=300)
        if success:
            self.repairs_applied.append("PyTorch installation repaired")
            return True
        else:
            print(f"❌ PyTorch repair failed: {output}")
            return False

    def auto_repair_dependencies(self):
        """Auto-repair missing dependencies"""
        print("🔧 Installing missing dependencies...")

        # Install core requirements
        requirements_cmd = [self.pip_cmd, "install", "-r", "requirements.txt"]
        success, output = self.run_command_safely(requirements_cmd, "Dependencies Installation", timeout=300)

        if success:
            self.repairs_applied.append("Dependencies installed")
            return True
        else:
            # Fallback: install critical packages individually
            critical_packages = [
                "gradio>=4.0.0", "transformers", "accelerate", "diffusers",
                "opencv-python", "Pillow", "numpy", "scipy"
            ]

            for package in critical_packages:
                cmd = [self.pip_cmd, "install", package]
                success, _ = self.run_command_safely(cmd, f"Installing {package}", timeout=60)
                if success:
                    print(f"✅ Installed {package}")

            self.repairs_applied.append("Critical dependencies installed individually")
            return True

    def auto_repair_wan2gp_repo(self):
        """Auto-repair WAN2GP repository"""
        print("🔧 Cloning WAN2GP repository...")

        repo_url = "https://github.com/deepbeepmeep/Wan2GP.git"
        clone_cmd = ["git", "clone", "--depth", "1", repo_url]

        success, output = self.run_command_safely(clone_cmd, "Repository Clone", timeout=120)
        if success:
            self.repairs_applied.append("WAN2GP repository cloned")
            return True
        else:
            print(f"❌ Repository clone failed: {output}")
            return False

    def auto_repair_attention_mechanisms(self):
        """Auto-repair attention mechanisms"""
        print("🔧 Installing performance optimizations...")

        repairs = []

        # Install Triton for Windows
        if os.name == 'nt':  # Windows
            triton_cmd = [self.pip_cmd, "install", "triton-windows"]
            success, _ = self.run_command_safely(triton_cmd, "Triton Installation", timeout=120)
            if success:
                repairs.append("Triton (Windows)")

        # Install SageAttention
        sage_cmd = [self.pip_cmd, "install", "sageattention>=1.0.6"]
        success, _ = self.run_command_safely(sage_cmd, "SageAttention Installation", timeout=120)
        if success:
            repairs.append("SageAttention")

        if repairs:
            self.repairs_applied.append(f"Installed: {', '.join(repairs)}")
            return True

        return False

    def generate_optimized_launch_commands(self):
        """Generate optimized launch commands based on hardware"""
        print("\n🚀 Generating optimized launch commands...")

        vram_gb = self.gpu_info.get('vram_gb', 8)
        gpu_name = self.gpu_info.get('name', 'Unknown')

        commands = {}

        # Base command components
        base_cmd = "python wgp.py"

        if vram_gb < 8:
            # Low VRAM setup
            commands['Low VRAM (6-8GB)'] = f"{base_cmd} --t2v-1-3B --attention sdpa --profile 4 --teacache 1.5"
        elif vram_gb < 12:
            # Medium VRAM setup
            commands['Medium VRAM (8-12GB)'] = f"{base_cmd} --t2v-14B --attention sage --profile 4 --teacache 2.0"
        else:
            # High VRAM setup
            commands['High VRAM (12GB+)'] = f"{base_cmd} --t2v-14B --attention sage2 --profile 3 --compile --teacache 2.0"

        # GPU-specific optimizations
        if "10" in gpu_name or "20" in gpu_name:
            commands['RTX 10XX/20XX Optimized'] = f"{base_cmd} --attention sdpa --profile 4 --teacache 1.5"
        elif "30" in gpu_name or "40" in gpu_name:
            commands['RTX 30XX/40XX Optimized'] = f"{base_cmd} --compile --attention sage --profile 3 --teacache 2.0"
        elif "50" in gpu_name:
            commands['RTX 50XX Optimized'] = f"{base_cmd} --attention sage --profile 4 --fp16"

        # Fallback command
        commands['Safe Fallback'] = f"{base_cmd} --t2v-1-3B --attention sdpa --profile 4 --teacache 0 --fp16"

        # Debug command
        commands['Debug Mode'] = f"{base_cmd} --verbose 2 --check-loras --attention sdpa --profile 4"

        return commands

    def run_full_diagnostic(self):
        """Run complete diagnostic and repair sequence"""
        print("🏥 WAN2GP Comprehensive Diagnostic and Auto-Repair System")
        print("=" * 60)
        print(f"🖥️ Platform: {self.platform}")
        print(f"🐍 Python: {self.python_cmd}")
        print(f"📦 Pip: {self.pip_cmd}")
        print(f"🔧 Virtual Environment: {'Yes' if self.use_venv else 'No'}")
        print("=" * 60)

        # Run all diagnostics
        checks = [
            self.check_python_version,
            self.check_gpu_compatibility,
            self.check_cuda_pytorch_compatibility,
            self.check_wan2gp_installation,
            self.check_dependencies,
            self.check_attention_mechanisms,
            self.check_memory_and_performance
        ]

        print("\n📋 Running Diagnostics...")
        for check in checks:
            try:
                check()
            except Exception as e:
                print(f"❌ Diagnostic error: {str(e)}")
                self.issues_found.append(f"Diagnostic error: {str(e)}")

        # Auto-repair if issues found
        if self.issues_found:
            print(f"\n⚠️ Found {len(self.issues_found)} issues:")
            for i, issue in enumerate(self.issues_found, 1):
                print(f"  {i}. {issue}")

            print(f"\n🔧 Attempting automatic repairs...")

            # Apply repairs based on issues found
            if any("PyTorch" in issue for issue in self.issues_found):
                self.auto_repair_pytorch()

            if any("dependencies" in issue.lower() for issue in self.issues_found):
                self.auto_repair_dependencies()

            if any("repository" in issue.lower() for issue in self.issues_found):
                self.auto_repair_wan2gp_repo()

            if any("attention" in issue.lower() for issue in self.issues_found):
                self.auto_repair_attention_mechanisms()

        # Generate launch commands
        commands = self.generate_optimized_launch_commands()

        # Final report
        self.display_final_report(commands)

    def display_final_report(self, commands):
        """Display comprehensive final report"""

        # Create styled HTML report
        html_report = f"""
        <div style="font-family: Arial, sans-serif; background: linear-gradient(135deg, #667eea 0%, #764ba2 100%);
                    color: white; padding: 20px; border-radius: 10px; margin: 10px 0;">
            <h2>🏥 WAN2GP Diagnostic Report</h2>
            <div style="background-color: rgba(255,255,255,0.1); padding: 15px; border-radius: 8px; margin: 10px 0;">
                <h3>📊 System Status</h3>
                <p><strong>Platform:</strong> {self.platform}</p>
                <p><strong>GPU:</strong> {self.gpu_info.get('name', 'Unknown')} ({self.gpu_info.get('vram_gb', 0):.1f}GB VRAM)</p>
                <p><strong>Issues Found:</strong> {len(self.issues_found)}</p>
                <p><strong>Repairs Applied:</strong> {len(self.repairs_applied)}</p>
            </div>
        </div>
        """

        if self.repairs_applied:
            html_report += f"""
            <div style="background-color: #28a745; color: white; padding: 15px; border-radius: 8px; margin: 10px 0;">
                <h3>✅ Repairs Applied Successfully:</h3>
                <ul>
            """
            for repair in self.repairs_applied:
                html_report += f"<li>{repair}</li>"
            html_report += "</ul></div>"

        if self.issues_found and not self.repairs_applied:
            html_report += f"""
            <div style="background-color: #dc3545; color: white; padding: 15px; border-radius: 8px; margin: 10px 0;">
                <h3>⚠️ Unresolved Issues:</h3>
                <ul>
            """
            for issue in self.issues_found:
                html_report += f"<li>{issue}</li>"
            html_report += "</ul></div>"

        display(HTML(html_report))

        # Display optimized commands
        print("\n🚀 Recommended Launch Commands:")
        print("=" * 50)
        for name, command in commands.items():
            print(f"\n{name}:")
            print(f"  {command}")

        print("\n" + "=" * 50)
        print("✅ Diagnostic complete! Use the appropriate command above to launch WAN2GP.")
        if self.issues_found and not self.repairs_applied:
            print("⚠️  Some issues require manual intervention. Check the report above.")

# Execute diagnostic system
diagnostic_system = WAN2GPDiagnosticAndRepair()
diagnostic_system.run_full_diagnostic()


🏥 WAN2GP Comprehensive Diagnostic and Auto-Repair System
🖥️ Platform: Google Colab
🐍 Python: .venv/bin/python
📦 Pip: .venv/bin/pip
🔧 Virtual Environment: Yes

📋 Running Diagnostics...
🔍 Checking Python version...
✅ Python version: 3.11.13
🔍 Checking GPU compatibility...
✅ GPU detected: 0  Tesla T4                       Off
✅ VRAM: 15.0GB
🔍 Checking CUDA and PyTorch compatibility...
✅ PyTorch version: 2.6.0+cu124
✅ CUDA available in PyTorch: True
🔍 Checking WAN2GP installation...
✅ WAN2GP found at: ../Wan2GP
🔍 Checking critical dependencies...
✅ PyTorch installed
✅ TorchVision installed
✅ Gradio installed
✅ Transformers installed
✅ Accelerate installed
✅ Diffusers installed
🔍 Checking attention mechanisms...
  ✅ Triton 3.2.0
  ❌ SageAttention not available
  ❌ Flash Attention not available
🔍 Checking memory and performance...
✅ System RAM: 12.7GB
✅ Free disk space: 17.8GB

⚠️ Found 3 issues:
  1. Python 3.11.13 detected. Python 3.10.9 recommended for best compatibility.
  2. Low system 


🚀 Recommended Launch Commands:

High VRAM (12GB+):
  python wgp.py --t2v-14B --attention sage2 --profile 3 --compile --teacache 2.0

Safe Fallback:
  python wgp.py --t2v-1-3B --attention sdpa --profile 4 --teacache 0 --fp16

Debug Mode:
  python wgp.py --verbose 2 --check-loras --attention sdpa --profile 4

✅ Diagnostic complete! Use the appropriate command above to launch WAN2GP.
⚠️  Some issues require manual intervention. Check the report above.


In [6]:
#@title Cell (2) - Quick Diagnostic Runner and Emergency Repair Tools (Latest v6.2)

import subprocess
import sys
import os
from IPython.display import display, HTML

class QuickDiagnosticRunner:
    def __init__(self):
        self.platform = self.detect_platform()
        self.pip_cmd, self.python_cmd = self.get_platform_commands()

    def detect_platform(self):
        """Quick platform detection"""
        if "google.colab" in sys.modules:
            return "Google Colab"
        elif any(indicator in str(sys.executable).lower() for indicator in ["lightning", "teamspace"]):
            return "Lightning AI"
        else:
            return "Generic/Vast.AI"

    def get_platform_commands(self):
        """Get platform-specific commands"""
        if self.platform == "Lightning AI":
            return "pip", "python"
        else:
            return ".venv/bin/pip" if os.path.exists(".venv") else "pip", ".venv/bin/python" if os.path.exists(".venv") else "python"

    def run_cmd(self, cmd, timeout=30):
        """Execute command safely"""
        try:
            result = subprocess.run(cmd, shell=True, capture_output=True, text=True, timeout=timeout)
            return result.returncode == 0, result.stdout, result.stderr
        except subprocess.TimeoutExpired:
            return False, "", "Timeout"
        except Exception as e:
            return False, "", str(e)

    def emergency_pytorch_fix(self):
        """Emergency PyTorch installation fix"""
        print("🚨 Emergency PyTorch Fix...")

        # Detect GPU generation for correct PyTorch version
        success, gpu_info, _ = self.run_cmd("nvidia-smi")

        if "RTX 50" in gpu_info:
            pytorch_url = "https://download.pytorch.org/whl/test/cu128"
            torch_version = "torch==2.7.0"
        else:
            pytorch_url = "https://download.pytorch.org/whl/test/cu124"
            torch_version = "torch==2.6.0"

        cmd = f"{self.pip_cmd} install --upgrade {torch_version} torchvision torchaudio --index-url {pytorch_url}"
        success, stdout, stderr = self.run_cmd(cmd, timeout=300)

        if success:
            print("✅ PyTorch emergency fix applied")
            return True
        else:
            print(f"❌ PyTorch fix failed: {stderr}")
            return False

    def emergency_dependency_fix(self):
        """Emergency dependency installation"""
        print("🚨 Emergency Dependency Fix...")

        essential_packages = [
            "gradio>=4.0.0", "transformers", "accelerate", "diffusers",
            "opencv-python", "Pillow", "numpy", "scipy", "psutil"
        ]

        for package in essential_packages:
            cmd = f"{self.pip_cmd} install {package}"
            success, _, _ = self.run_cmd(cmd, timeout=60)
            print("✅" if success else "❌", package)

        print("✅ Emergency dependencies installed")

    def quick_system_check(self):
        """Quick system health check"""
        checks = {}

        # GPU Check
        success, output, _ = self.run_cmd("nvidia-smi")
        checks['GPU'] = "✅ Available" if success and "RTX" in output else "❌ Issue detected"

        # PyTorch Check
        try:
            import torch
            checks['PyTorch'] = f"✅ {torch.__version__}" if torch.cuda.is_available() else "❌ CUDA not available"
        except ImportError:
            checks['PyTorch'] = "❌ Not installed"

        # WAN2GP Check
        wan_exists = any(os.path.exists(path) for path in ["Wan2GP/wgp.py", "wan2gp/wgp.py", "WAN2GP/wgp.py"])
        checks['WAN2GP'] = "✅ Found" if wan_exists else "❌ Missing"

        # Dependencies Check
        try:
            import gradio, transformers, accelerate, diffusers
            checks['Dependencies'] = "✅ Core packages available"
        except ImportError:
            checks['Dependencies'] = "❌ Missing packages"

        return checks

    def generate_emergency_commands(self):
        """Generate emergency launch commands"""
        commands = {
            "Ultra Safe Mode": f"{self.python_cmd} wgp.py --t2v-1-3B --attention sdpa --profile 4 --teacache 0 --fp16",
            "Memory Emergency": f"{self.python_cmd} wgp.py --t2v-1-3B --profile 5 --perc-reserved-mem-max 0.2",
            "Debug Mode": f"{self.python_cmd} wgp.py --verbose 2 --attention sdpa --profile 4",
            "Network Share": f"{self.python_cmd} wgp.py --listen --server-port 7861 --attention sdpa"
        }
        return commands

    def run_quick_diagnostic(self):
        """Run quick diagnostic and provide emergency options"""

        print(f"⚡ Quick Diagnostic - Platform: {self.platform}")
        print("=" * 40)

        # Quick system check
        checks = self.quick_system_check()

        issues = []
        for component, status in checks.items():
            print(f"{component}: {status}")
            if "❌" in status:
                issues.append(component)

        # Emergency repairs
        if issues:
            print(f"\n🚨 {len(issues)} issues detected. Applying emergency fixes...")

            if "PyTorch" in issues:
                self.emergency_pytorch_fix()

            if "Dependencies" in issues:
                self.emergency_dependency_fix()

            if "WAN2GP" in issues:
                print("🔧 Cloning WAN2GP repository...")
                success, _, _ = self.run_cmd("git clone --depth 1 https://github.com/deepbeepmeep/Wan2GP.git", timeout=120)
                print("✅ Repository cloned" if success else "❌ Clone failed")

        # Generate emergency commands
        commands = self.generate_emergency_commands()

        # Display results
        html_display = f"""
        <div style="background: linear-gradient(45deg, #ff6b6b, #feca57); color: white;
                    padding: 15px; border-radius: 8px; margin: 10px 0;">
            <h3>⚡ Quick Diagnostic Results</h3>
            <p><strong>Platform:</strong> {self.platform}</p>
            <p><strong>Issues Found:</strong> {len(issues)}</p>
            <p><strong>Status:</strong> {'🚨 Needs Attention' if issues else '✅ System Ready'}</p>
        </div>
        """

        display(HTML(html_display))

        print("\n🚀 Emergency Launch Commands:")
        print("-" * 40)
        for name, command in commands.items():
            print(f"\n{name}:")
            print(f"  {command}")

        print("\n" + "=" * 40)
        print("⚡ Quick diagnostic complete!")

        if not issues:
            print("✅ System appears healthy. Try the Ultra Safe Mode command first.")
        else:
            print("🚨 Emergency fixes applied. Test with Ultra Safe Mode.")

# Execute quick diagnostic
quick_runner = QuickDiagnosticRunner()
quick_runner.run_quick_diagnostic()


⚡ Quick Diagnostic - Platform: Google Colab
GPU: ❌ Issue detected
PyTorch: ✅ 2.6.0+cu124
WAN2GP: ❌ Missing
Dependencies: ✅ Core packages available

🚨 2 issues detected. Applying emergency fixes...
🔧 Cloning WAN2GP repository...
✅ Repository cloned



🚀 Emergency Launch Commands:
----------------------------------------

Ultra Safe Mode:
  .venv/bin/python wgp.py --t2v-1-3B --attention sdpa --profile 4 --teacache 0 --fp16

Memory Emergency:
  .venv/bin/python wgp.py --t2v-1-3B --profile 5 --perc-reserved-mem-max 0.2

Debug Mode:
  .venv/bin/python wgp.py --verbose 2 --attention sdpa --profile 4

Network Share:
  .venv/bin/python wgp.py --listen --server-port 7861 --attention sdpa

⚡ Quick diagnostic complete!
🚨 Emergency fixes applied. Test with Ultra Safe Mode.
